In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Load data
data = pd.read_csv('https://raw.githubusercontent.com/huy164/datasets/master/walmart_stock_price.csv')
dates = data['Date'].values.astype(np.datetime64)
prices = data['Close'].values.astype('float32')

# Split data into train, validation, and test sets
train_size = int(0.6 * len(prices))
val_size = int(0.2 * len(prices))
test_size = len(prices) - train_size - val_size
train_data = prices[:train_size].reshape(-1, 1)
val_data = prices[train_size:train_size+val_size].reshape(-1, 1)
test_data = prices[train_size+val_size:].reshape(-1, 1)

# Normalize data
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
val_data = scaler.transform(val_data)
test_data = scaler.transform(test_data)

# Define sequence length and number of features
seq_length = 30
num_features = 1

# Create sequences
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data)-seq_length-1):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_data, seq_length)
X_val, y_val = create_sequences(val_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

# Define model architecture
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(64, input_shape=(seq_length, num_features)),
    tf.keras.layers.Dense(1)
])

# Compile model
model.compile(optimizer='adam', loss='mse')

# Train model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=64)

# Evaluate model
loss = model.evaluate(X_test, y_test)
print('Loss: ', loss)

train_predictions = scaler.inverse_transform(model.predict(X_train))
val_predictions = scaler.inverse_transform(model.predict(X_val))
test_predictions = scaler.inverse_transform(model.predict(X_test))

Epoch 1/50
117/117 [==============================] - 6s 26ms/step - loss: 0.0013 - val_loss: 1.6611e-04
Epoch 2/50
117/117 [==============================] - 3s 22ms/step - loss: 7.6999e-05 - val_loss: 3.4409e-04
Epoch 3/50
117/117 [==============================] - 4s 33ms/step - loss: 7.4072e-05 - val_loss: 3.2646e-04
Epoch 4/50
117/117 [==============================] - 2s 20ms/step - loss: 7.5537e-05 - val_loss: 1.7422e-04
Epoch 5/50
117/117 [==============================] - 2s 21ms/step - loss: 7.5644e-05 - val_loss: 1.3086e-04
Epoch 6/50
117/117 [==============================] - 2s 20ms/step - loss: 6.7125e-05 - val_loss: 1.6235e-04
Epoch 7/50
117/117 [==============================] - 2s 16ms/step - loss: 7.6265e-05 - val_loss: 1.2580e-04
Epoch 8/50
117/117 [==============================] - 1s 13ms/step - loss: 8.4191e-05 - val_loss: 1.4253e-04
Epoch 9/50
117/117 [==============================] - 2s 17ms/step - loss: 6.5118e-05 - val_loss: 1.4581e-04
Epoch 10/50
117/117 [==